# Sentinel 2 resources on AWS

- Sentinel 2 resources on AWS: [Sentinel-2 - Registry of Open Data on AWS](https://registry.opendata.aws/sentinel-2/)
- Data managed by Sinergy on AWS: [S2 on AWS from Sinergy](https://registry.opendata.aws/?search=managedBy:sinergise)


Question: verify the existence of the Sentinel-2, production line 500, on AWS
Procedure description:

- Download the latest manifest file from the Sentinel-2 registry on AWS
- Load the manifest file into a pandas dataframe
- Display the last 3 rows of the dataframe
- Download the latest inventory file from the Sentinel-2 registry on AWS
- Check the last 10 rows of the inventory file

Source of data: 
```
Description
    S3 Inventory files for L2A (ORC and CSV)
Resource type
    S3 Bucket
Amazon Resource Name (ARN)
    arn:aws:s3:::sentinel-inventory/sentinel-s2-l2a
AWS Region
    eu-central-1
AWS CLI Access (No AWS account required)
    aws s3 ls --no-sign-request s3://sentinel-inventory/sentinel-s2-l2a/
```

In [25]:
import boto3 
import pandas as pd
import subprocess

In [22]:
# get latest date to retreat manifest file
!aws s3 ls --no-sign-request s3://sentinel-inventory/sentinel-s2-l2a/sentinel-s2-l2a-inventory/

                           PRE 2018-09-08T08-00Z/
                           PRE 2018-09-09T08-00Z/
                           PRE 2018-09-10T08-00Z/
                           PRE 2018-09-11T08-00Z/
                           PRE 2018-09-12T08-00Z/
                           PRE 2018-09-13T08-00Z/
                           PRE 2018-09-14T08-00Z/
                           PRE 2018-09-15T08-00Z/
                           PRE 2018-09-16T08-00Z/
                           PRE 2018-09-17T08-00Z/
                           PRE 2018-09-18T08-00Z/
                           PRE 2018-09-19T08-00Z/
                           PRE 2018-09-20T08-00Z/
                           PRE 2018-09-21T08-00Z/
                           PRE 2018-09-22T08-00Z/
                           PRE 2018-09-23T08-00Z/
                           PRE 2018-09-24T08-00Z/
                           PRE 2018-09-25T08-00Z/
                           PRE 2018-09-26T08-00Z/
                           PRE 2018-09-27T08-00Z/


In [23]:
# get latest manifest file
!aws s3 cp --no-sign-request s3://sentinel-inventory/sentinel-s2-l2a/sentinel-s2-l2a-inventory/2024-02-11T01-00Z/manifest.json ./data/

Completed 256.0 KiB/347.4 KiB (700.1 KiB/s) with 1 file(s) remaining
Completed 347.4 KiB/347.4 KiB (921.1 KiB/s) with 1 file(s) remaining
download: s3://sentinel-inventory/sentinel-s2-l2a/sentinel-s2-l2a-inventory/2024-02-11T01-00Z/manifest.json to data\manifest.json


In [39]:
pd.read_json(r'./data/manifest.json').tail(3)

,sourceBucket,destinationBucket,version,creationTimestamp,fileFormat,fileSchema,files
1863,sentinel-s2-l2a,arn:aws:s3:::sentinel-inventory,2016-11-30,1707613200000,CSV,"Bucket, Key, Size, LastModifiedDate",{'key': 'sentinel-s2-l2a/sentinel-s2-l2a-inven...
1864,sentinel-s2-l2a,arn:aws:s3:::sentinel-inventory,2016-11-30,1707613200000,CSV,"Bucket, Key, Size, LastModifiedDate",{'key': 'sentinel-s2-l2a/sentinel-s2-l2a-inven...
1865,sentinel-s2-l2a,arn:aws:s3:::sentinel-inventory,2016-11-30,1707613200000,CSV,"Bucket, Key, Size, LastModifiedDate",{'key': 'sentinel-s2-l2a/sentinel-s2-l2a-inven...


In [47]:
# get latest time stamp to retreat inventory file
pd.to_datetime(pd.read_json(r'./data/manifest.json').iloc[-1]['creationTimestamp'], unit='ms', origin='unix')

Timestamp('2024-02-11 01:00:00')

In [30]:
path = pd.read_json(r'./data/manifest.json').iloc[-1].files['key']

In [31]:
subprocess.run(["aws", "s3", "cp", "--no-sign-request", f"s3://sentinel-inventory/{path}", "./data/"])

CompletedProcess(args=['aws', 's3', 'cp', '--no-sign-request', 's3://sentinel-inventory/sentinel-s2-l2a/sentinel-s2-l2a-inventory/data/be33757d-19f9-4c32-9519-c5e363822cbd.csv.gz', './data/'], returncode=0)

In [37]:
file_path = path.split("/")[-1]
pd.read_csv(rf"./data/{file_path}").tail(10)

,sentinel-s2-l2a,products/2021/9/21/S2B_MSIL2A_20210921T170919_N0301_R112_T15STD_20210921T214605/datastrip/0/qi/GEOMETRIC_QUALITY.xml,7452,2021-09-22T00:42:47.000Z
999989,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,8278,2023-06-24T03:47:52.000Z
999990,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,7229,2023-06-24T03:47:52.000Z
999991,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,4472,2023-06-24T03:47:52.000Z
999992,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,19430,2023-06-24T03:47:52.000Z
999993,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,69647,2023-06-24T03:47:52.000Z
999994,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,55561,2023-06-24T03:47:52.000Z
999995,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,1035,2023-06-24T03:47:52.000Z
999996,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,20864374,2023-07-06T13:32:01.000Z
999997,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,4041,2023-07-06T13:32:01.000Z
999998,sentinel-s2-l2a,products/2021/9/26/S2A_MSIL2A_20210926T020701_...,5828,2023-07-06T13:32:01.000Z


# conclusion 

AWS host all the data from Sentinel-2 MSI L2A, production line 500 in the region `eu-central-1`. 